# Lab 3

In [5]:
import sqlite3

# Init db

In [6]:
# connect to db
conn = sqlite3.connect('lab_3.db')
cursor = conn.cursor()

# Create tables

In [7]:
# create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Professors (
    prof_ssn CHAR(10) PRIMARY KEY,
    name CHAR(64),
    age INTEGER,
    rank INTEGER,
    speciality CHAR(64)
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Depts (
    dno INTEGER PRIMARY KEY,
    dname CHAR(64),
    office CHAR(10)
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Runs (
    dno INTEGER,
    prof_ssn CHAR(10),
    PRIMARY KEY (dno, prof_ssn),
    FOREIGN KEY (prof_ssn) REFERENCES Professors,
    FOREIGN KEY (dno) REFERENCES Depts
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Work_Dept (
    dno INTEGER,
    prof_ssn CHAR(10),
    pc_time INTEGER,
    PRIMARY KEY (dno, prof_ssn),
    FOREIGN KEY (prof_ssn) REFERENCES Professors,
    FOREIGN KEY (dno) REFERENCES Depts
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Projects (
    pid INTEGER PRIMARY KEY,
    sponsor CHAR(32),
    start_date DATE,
    end_date DATE,
    budget FLOAT
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Graduates (
    grad_ssn CHAR(10) PRIMARY KEY,
    age INTEGER,
    name CHAR(64),
    deg_prog CHAR(32),
    major INTEGER,
    FOREIGN KEY (major) REFERENCES Depts
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Advisor (
    senior_ssn CHAR(10),
    grad_ssn CHAR(10),
    PRIMARY KEY (senior_ssn, grad_ssn),
    FOREIGN KEY (senior_ssn) REFERENCES Graduates(grad_ssn),
    FOREIGN KEY (grad_ssn) REFERENCES Graduates
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Manages (
    pid INTEGER,
    prof_ssn CHAR(10),
    PRIMARY KEY (pid, prof_ssn),
    FOREIGN KEY (prof_ssn) REFERENCES Professors,
    FOREIGN KEY (pid) REFERENCES Projects
);''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Work_In (
    pid INTEGER,
    prof_ssn CHAR(10),
    PRIMARY KEY (pid, prof_ssn),
    FOREIGN KEY (prof_ssn) REFERENCES Professors,
    FOREIGN KEY (pid) REFERENCES Projects
);''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS Supervises (
    prof_ssn CHAR(10),
    grad_ssn CHAR(10),
    pid INTEGER,
    PRIMARY KEY (prof_ssn, grad_ssn, pid),
    FOREIGN KEY (prof_ssn) REFERENCES Professors,
    FOREIGN KEY (grad_ssn) REFERENCES Graduates,
    FOREIGN KEY (pid) REFERENCES Projects
);''')


# Add data

In [8]:
# add data
professors = [
    ('P1', 'Prof. Smith', 55, 1, 'AI'),
    ('P2', 'Prof. Johnson', 45, 2, 'DBMS'),
    ('P3', 'Prof. Lee', 50, 1, 'Networking'),
    ('P4', 'Prof. Kim', 40, 3, 'Cybersecurity')
]
cursor.executemany('INSERT OR IGNORE INTO Professors VALUES (?, ?, ?, ?, ?)', professors)

depts = [
    (1, 'Computer Science', 'A101'),
    (2, 'Mathematics', 'B201'),
    (3, 'Physics', 'C301')
]
cursor.executemany('INSERT OR IGNORE INTO Depts VALUES (?, ?, ?)', depts)


runs = [
    (1, 'P1'),
    (2, 'P2'),
    (3, 'P3')
]
cursor.executemany('INSERT OR IGNORE INTO Runs VALUES (?, ?)', runs)


work_dept = [
    (1, 'P1', 30),
    (2, 'P2', 20),
    (3, 'P3', 25)
]
cursor.executemany('INSERT OR IGNORE INTO Work_Dept VALUES (?, ?, ?)', work_dept)


projects = [
    (101, 'NSF', '2023-01-01', '2023-12-31', 50000),
    (102, 'Google', '2022-06-01', '2024-05-31', 100000),
    (103, 'Microsoft', '2023-03-01', '2023-11-30', 75000)
]
cursor.executemany('INSERT OR IGNORE INTO Projects VALUES (?, ?, ?, ?, ?)', projects)


graduates = [
    ('G1', 24, 'Alice', 'M.Sc', 1),
    ('G2', 26, 'Bob', 'Ph.D', 2),
    ('G3', 23, 'Charlie', 'B.Sc', 3)
]
cursor.executemany('INSERT OR IGNORE INTO Graduates VALUES (?, ?, ?, ?, ?)', graduates)


advisors = [
    ('G2', 'G1'),
    ('G3', 'G2')
]
cursor.executemany('INSERT OR IGNORE INTO Advisor VALUES (?, ?)', advisors)


manages = [
    (101, 'P1'),
    (102, 'P2'),
    (103, 'P3')
]
cursor.executemany('INSERT OR IGNORE INTO Manages VALUES (?, ?)', manages)


work_in = [
    (101, 'P1'),
    (102, 'P2'),
    (103, 'P3')
]
cursor.executemany('INSERT OR IGNORE INTO Work_In VALUES (?, ?)', work_in)


supervises = [
    ('P1', 'G1', 101),
    ('P2', 'G2', 102),
    ('P3', 'G3', 103)
]
cursor.executemany('INSERT OR IGNORE INTO Supervises VALUES (?, ?, ?)', supervises)

conn.commit()


**Section 2: Joins**
#### **Examples**
1. Retrieve all professors and their departments.
   ```sql
   SELECT p.name AS Professor, d.dname AS Department
   FROM Professors p
   JOIN Runs r ON p.prof_ssn = r.prof_ssn
   JOIN Depts d ON r.dno = d.dno;
   ```
2. Find the graduates and their advisors.
   ```sql
   SELECT g.name AS Graduate, g2.name AS Advisor
   FROM Graduates g
   JOIN Advisor a ON g.grad_ssn = a.grad_ssn
   JOIN Graduates g2 ON a.senior_ssn = g2.grad_ssn;
    ```
#### Questions:
3. List the names of all professors and their associated projects.
4. List all graduates along with their advisors and the department they are majoring in.
5. Retrieve professors who manage at least one project and the project details.

In [14]:
# q3
results = cursor.execute("""
SELECT Professors.name, Projects.pid FROM Professors JOIN Supervises ON Professors.prof_ssn = Supervises.prof_ssn JOIN Projects ON Supervises.pid = Projects.pid

""")
print("Professor Name\tProject ID")
for row in results.fetchall():
    print(f"{row[0]}\t{row[1]}")

Professor Name	Project ID
Prof. Smith	101
Prof. Johnson	102
Prof. Lee	103


In [20]:
# q4
results = cursor.execute("""
SELECT g1.name as student, g2.name as advisor, d.dname as department FROM Graduates g1 JOIN Advisor a ON g1.grad_ssn = a.grad_ssn JOIN Graduates g2 ON a.senior_ssn = g2.grad_ssn JOIN Depts d ON g1.major = d.dno;

""")
print("Graduate\tAdvisor\tDepartment")
for row in results.fetchall():
    print(f"{row[0]}\t{row[1]}\t{row[2]}")

Graduate	Advisor	Department
Alice	Bob	Computer Science
Bob	Charlie	Mathematics


In [31]:
# q5
results = cursor.execute("""
SELECT Professors.name, Projects.* FROM Professors JOIN Supervises ON Professors.prof_ssn = Supervises.prof_ssn JOIN Projects ON Projects.pid = Supervises.pid;

""")
print("Professor Name\t\tProject ID\tSponsor\t\tStart Date\t\tEnd Date\t\tBudget")
for row in results.fetchall():
    print(f"{row[0]}\t\t{row[1]}\t\t{row[2]}\t\t{row[3]}\t\t{row[4]}\t\t{row[5]}")

Professor Name		Project ID	Sponsor		Start Date		End Date		Budget
Prof. Smith		101		NSF		2023-01-01		2023-12-31		50000.0
Prof. Johnson		102		Google		2022-06-01		2024-05-31		100000.0
Prof. Lee		103		Microsoft		2023-03-01		2023-11-30		75000.0




### **Section 2: Nested Queries**

#### **Examples**
1. Find the names of professors who manage projects with budgets greater than $70,000.
   ```sql
   SELECT name
   FROM Professors
   WHERE prof_ssn IN (
       SELECT prof_ssn
       FROM Manages
       WHERE pid IN (
           SELECT pid
           FROM Projects
           WHERE budget > 70000
       )
   );
   ```

2. Retrieve graduates who have the same major as Prof. Smith's department.
   ```sql
   SELECT name
   FROM Graduates
   WHERE major = (
       SELECT dno
       FROM Runs
       WHERE prof_ssn = 'P1'
   );
   ```

#### **Questions**
3. Find graduates whose advisor is supervising atleast two students.
4. Retrieve the name of the professor who manages the project with the highest budget.
5. List all graduates who are majoring in the same department as their advisors.


In [51]:
# q3
results = cursor.execute("""
SELECT g1.name as student FROM Graduates g1 JOIN Advisor a ON g1.grad_ssn = a.grad_ssn
WHERE a.senior_ssn IN (
SELECT senior_ssn FROM Advisor GROUP BY senior_ssn HAVING COUNT(*) >= 2
);

""")
print(results.fetchall())

[]


In [43]:
# q4
results = cursor.execute("""
SELECT Professors.name FROM Professors JOIN Supervises ON Professors.prof_ssn = Supervises.prof_ssn JOIN Projects ON Projects.pid = Supervises.pid WHERE Projects.budget = (SELECT MAX(budget) FROM Projects)

""")
print(results.fetchall())

[('Prof. Johnson',)]


In [46]:
# q5
results = cursor.execute("""
SELECT g1.name as student FROM Graduates g1 JOIN Advisor a ON g1.grad_ssn = a.grad_ssn JOIN Graduates g2 ON a.senior_ssn = g2.grad_ssn
WHERE g1.major = g2.major;
""")
print(results.fetchall())

[]


### **Section 3: Triggers**

#### **Examples**
1. Prevent insertion of a graduate younger than 21.
   ```sql
   CREATE TRIGGER prevent_young_grads
   BEFORE INSERT ON Graduates
   FOR EACH ROW
   BEGIN
       SELECT CASE
           WHEN NEW.age < 21 THEN
               RAISE(ABORT, 'Graduate must be at least 21 years old')
       END;
   END;
   ```

2. Automatically assign a professor to a department if they are added without one.
   ```sql
   CREATE TRIGGER assign_default_dept
   AFTER INSERT ON Professors
   FOR EACH ROW
   WHEN NOT EXISTS (SELECT * FROM Runs WHERE prof_ssn = NEW.prof_ssn)
   BEGIN
       INSERT INTO Runs (dno, prof_ssn) VALUES (1, NEW.prof_ssn);
   END;
   ```

#### **Questions**
3. Write a trigger to ensure that projects with a budget over $100,000 cannot be added.
4. Create a trigger to ensure that when a project is deleted, all corresponding entries in Manages and Work_In are also deleted.
5. Create a trigger to automatically assign a default advisor to a graduate if none is provided.

In [48]:
# q3
results = cursor.execute("""
CREATE TRIGGER prevent_big_budget
BEFORE INSERT ON Projects
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN NEW.budget > 100000 THEN
            RAISE(ABORT, "The project's budget must be less than or $100,000")
    END;
END;
""")
print(results.fetchall())

[]


In [49]:
# q4
results = cursor.execute(
"""
CREATE TRIGGER cascade_delete_project
AFTER DELETE ON Projects
FOR EACH ROW
BEGIN
    DELETE FROM Manages WHERE pid = OLD.pid;
    DELETE FROM Work_in WHERE pid = OLD.pid;
END; 

""")
print(results.fetchall())

[]


In [50]:
# q5
results = cursor.execute("""
CREATE TRIGGER assign_advisor
AFTER INSERT ON Graduates
FOR EACH ROW
WHEN NOT EXISTS (SELECT * FROM Advisor WHERE grad_ssn = NEW.grad_ssn)
BEGIN
    INSERT INTO Advisor(senior_ssn, grad_ssn) VALUES ('G1', NEW.grad_ssn);
END;
""")
print(results.fetchall())

[]
